# **Strain Gage Calculations - v1**

- All the required input files should be in the same directory as this notebook
- Output files will be in the same directory as well.

--------
#### **Import Libraries**

In [ ]:
import pandas as pd
import numpy as np
import re
import plotly.express as px
import plotly as plt
from openpyxl import load_workbook
# import sys
# from PyQt5.QtWidgets import QApplication, QFileDialog

------

#### **Define Material Properties**

In [ ]:
# Define the material properties and the strain gauge angles

E = 200e9  # Young's Modulus in Pascals (Pa)
v = 0.3  # Poisson's ratio

--------
#### **Functions for Converting the Measured Rosette Strains into Various Outputs** 

References for the formulas used: 

https://community.sw.siemens.com/s/article/rosette-strain-gauges

https://www.youtube.com/watch?v=9x-3H74a8YQ


In [ ]:
def transform_strains_to_global(epsilon_A, epsilon_B, epsilon_C, angles):
    """
    Transform strains from the rosette's local coordinate system to the global coordinate system.

    Parameters:
    - epsilon_A (float): Measured normal strain from the first strain gauge in the rosette.
    - epsilon_B (float): Measured normal strain from the second strain gauge in the rosette.
    - epsilon_C (float): Measured normal strain from the third strain gauge in the rosette.
    - angles (list of float): The angles (in degrees) of the strain gauges relative to the global x-axis.

    Returns:
    - numpy.array: The transformed strains in the global coordinate system, containing
      the normal strains epsilon_x and epsilon_y, along with the engineering shear strain gamma_xy.
    """
    theta_A, theta_B, theta_C = np.radians(angles)
    T = np.array([
        [np.cos(theta_A)**2, np.sin(theta_A)**2, 1 * np.sin(theta_A) * np.cos(theta_A)],
        [np.cos(theta_B)**2, np.sin(theta_B)**2, 1 * np.sin(theta_B) * np.cos(theta_B)],
        [np.cos(theta_C)**2, np.sin(theta_C)**2, 1 * np.sin(theta_C) * np.cos(theta_C)]
    ])
    T_inv = np.linalg.inv(T)
    local_strains = np.array([epsilon_A, epsilon_B, epsilon_C])
    global_strains = T_inv @ local_strains
    return global_strains

def calculate_principal_strains(epsilon_x, epsilon_y, gamma_xy):
    """
    Calculate the principal strains using Mohr's circle relations.

    Parameters:
    - epsilon_x (float): Normal strain in the x-direction (global coordinate system).
    - epsilon_y (float): Normal strain in the y-direction (global coordinate system).
    - gamma_xy (float): Engineering shear strain in the global coordinate system.

    Returns:
    - numpy.array: An array containing the maximum (epsilon_1) and minimum (epsilon_2)
      principal strains. These values are critical for assessing the material's behavior
      under stress and for failure analysis.

    Note:
    - The shear strain input should be in engineering terms (total angular deformation),
      as this is the standard output from strain gauges.
    """
    C = (epsilon_x + epsilon_y) / 2
    R = np.sqrt(((epsilon_x - epsilon_y) / 2)**2 + (gamma_xy / 2)**2)
    epsilon_1 = C + R  # Maximum principal strain
    epsilon_2 = C - R  # Minimum principal strain
    return np.array([epsilon_1, epsilon_2])

def calculate_principal_stresses(principal_strains, E, v):
    """
    Calculate the principal stresses from the principal strains using material properties.

    Given the principal strains, this function applies Hooke's law in two dimensions to
    compute the principal stresses. The material's Young's modulus (E) and Poisson's ratio (v)
    are used to relate the strains to the stresses. This function assumes a linear elastic
    material behavior and plane stress conditions, which is a common scenario in thin structures
    where one dimension is significantly smaller than the other two.

    Parameters:
    - principal_strains (numpy.array): An array of the principal strains [epsilon_1, epsilon_2].
    - E (float): Young's Modulus of the material in Pascals (Pa).
    - v (float): Poisson's ratio of the material, dimensionless.

    Returns:
    - numpy.array: An array containing the principal stresses [sigma_1, sigma_2] in Pascals (Pa).
      These stresses are the maximum and minimum normal stresses that occur at the principal
      strain orientations, where the shear stress is zero.

    Note:
    - The principal strains should be provided in microstrains (με) for the calculation.
    - The function returns the principal stresses in Pascals (Pa), but they are manually
      converted to other MPa (MegaPascals) at the function return (by a /1e6 division).
    - The function holds true for isotropic materials where the stress-strain relationship 
      is governed by the isotropic form of Hooke's law.
    """
    #S = np.array([
    #    [1, v, 0],
    #    [v, 1, 0],
    #    [0, 0, (1-v)/2]
    #]) * E / (1 - v**2)
    S = np.array([
        [1, v],
        [v, 1]
    ]) * E / (1 - v**2)

    principal_stresses = S @ (principal_strains /1e6)
    return principal_stresses / 1e6  # Convert Pa to MPa

def calculate_principal_strain_orientation(epsilon_x, epsilon_y, gamma_xy):
    """
    Calculate the orientation of the principal strains from the original strain measurements using Mohr's circle analysis.
    Parameters:
    - epsilon_x (float): Normal strain in the x-direction.
    - epsilon_y (float): Normal strain in the y-direction.
    - gamma_xy (float): Engineering shear strain.
    Returns:
    - theta_p (float): Angle of the principal strain in degrees.
    """
    # Calculate the angle to the maximum principal strain
    theta_p_rad = 0.5 * np.arctan2(gamma_xy, epsilon_x - epsilon_y)
    theta_p = np.degrees(theta_p_rad)

    # Adjust the angle to ensure it's within the 0-180 degree range
    if theta_p < 0:
        theta_p += 180

    return theta_p

def calculate_biaxiality_ratio(S1, S2):
    """
    Calculate the biaxiality ratio from the principal stresses.

    The biaxiality ratio is the ratio of the second principal stress to the first principal stress.
    It is a dimensionless number that indicates the relation between the principal stresses. A
    biaxiality ratio of 1 indicates equal biaxial stress state, while a value of 0 indicates a uniaxial
    stress state. Negative values indicate that the principal stresses are of opposite signs.

    Parameters:
    - S1 (float or numpy.array): First principal stress (assumed to be the larger one in absolute value).
    - S2 (float or numpy.array): Second principal stress (assumed to be the smaller one in absolute value).

    Returns:
    - float or numpy.array: The biaxiality ratio, a dimensionless quantity indicating the biaxial state of stress.

    Note:
    - The function assumes that S1 and S2 are provided such that the absolute value of S1 is
      greater than or equal to the absolute value of S2. If this is not the case, the inputs
      should be swapped.
    """
    # Ensure that sigma_1 is the larger one in absolute terms
    sigma_1 = np.where(np.abs(S1) >= np.abs(S2), S1, S2)
    sigma_2 = np.where(np.abs(S1) >= np.abs(S2), S2, S1)
    
    # Calculate the biaxiality ratio
    biaxiality_ratio = sigma_2 / sigma_1
    
    return biaxiality_ratio

def calculate_von_mises_stress(S1, S2, S3=0):
    """
    Calculate the von Mises stress from the principal stresses.
    
    Parameters:
    S1 (float): First principal stress in MPa.
    S2 (float): Second principal stress in MPa.
    S3 (float): Third principal stress in MPa, assumed to be zero for plane stress condition.
    
    Returns:
    float: The von Mises stress in MPa.
    """
    # Calculate the von Mises stress using the principal stresses
    sigma_vm = np.sqrt(((S1 - S2)**2 + (S1 - S3)**2 + (S2 - S3)**2) / 2)
    return sigma_vm


-------

#### **Load Input Files**

>**SG_sample_out_data_sg_filtered_v2_long_test.csv**
* Specifies the input CSV data

    - First column should be time in seconds
    
    - Remaining columns are strains in each rosette channel, (units are in [μmm/mm])

>**rosette_angles.csv**
* Specifies the angles of each channel of a rosette

    - First column should be the channel name of rosette (SG1_1, SG2 etc.)

    - Remaining columns should be angles in degrees wrt to preferred axis

In [ ]:
# Reading the files

# app = QApplication(sys.argv)
# file_path, _ = QFileDialog().getOpenFileName(None, 'Open File', '', 'All Files (*);;Text Files (*.txt)')

# if file_path:
#     print("Selected test data:", file_path)

# app = QApplication(sys.argv)
# angles_file_path, _ = QFileDialog().getOpenFileName(None, 'Open File', '', 'All Files (*);;Text Files (*.txt)')

# if angles_file_path:
#     print("Selected rosette angles data:", angles_file_path)

# Load the CSV file into a pandas DataFrame
#file_path = 'SG_sample_out_data_sg_filtered_v3_long_test.csv'
file_path = 'SG_sample_out_data_measured.csv'
data = pd.read_csv(file_path)

if "_measured" in file_path:
    data.drop([0,2,3,4,5,6], inplace=True)
    data.columns=data.iloc[0]
    new_columns = data.columns.tolist()
    new_columns[1] = 'Time'
    data.columns = new_columns
    data.drop([1], inplace=True)
    data.drop(data.columns[0], axis=1, inplace=True)

    # data.columns=data.iloc[0]
    # data.columns[2] = ['Time']
    # data = data[1:]
    # data

print("Selected test data:", file_path)

# Load the CSV file containing the rosette angles
angles_file_path = 'rosette_angles_v2.csv'
rosette_angles_df = pd.read_csv(angles_file_path)
print("Selected rosette angles data:", angles_file_path)

time = data['Time']
strain_gauge_data = data.iloc[:, 1:].filter(regex='SG')
strain_gauge_data = strain_gauge_data.astype(float)

----
#### **Main Calculation Loop**

In [ ]:
# Get a list of column names that match the pattern 'SG(\d+)_'
matching_columns = [col for col in strain_gauge_data.columns if re.search(r'SG(\d+)_', col)]
# Extract the numerical part from the matching column names
sg_numbers = [int(re.search(r'SG(\d+)_', col).group(1)) for col in matching_columns]
# Remove duplicates and sort the extracted numerical values
sg_numbers = sorted(set(sg_numbers))

for sg_number in sg_numbers:
    sg_cols = [col for col in strain_gauge_data.columns if f'SG{sg_number}_' in col]

    # Ensure there are three columns per strain gauge
    if len(sg_cols) == 3:
        rosette_row = rosette_angles_df[rosette_angles_df['SG'] == sg_number]
        if not rosette_row.empty:
            current_angles = rosette_row.iloc[0, 1:].values
            global_strains = np.array([transform_strains_to_global(*strain, current_angles) for strain in strains])
            principal_strains = np.array([calculate_principal_strains(strain[0], strain[1], strain[2]) for strain in global_strains])
            principal_stresses = np.array([calculate_principal_stresses(strain, E, v) for strain in principal_strains])
            principal_strain_orientation = np.array([calculate_principal_strain_orientation(strain[0], strain[1], strain[2]) for strain in global_strains])
            biaxiality_ratios = calculate_biaxiality_ratio(principal_stresses[:, 0], principal_stresses[:, 1])
            von_mises_stresses = np.array([calculate_von_mises_stress(*stress) for stress in principal_stresses])
            for i, strain_type in enumerate(['epsilon_x [με]', 'epsilon_y [με]', 'gamma_xy [με]']):
                strain_gauge_data[f'SG{sg_number}_{strain_type}'] = global_strains[:, i]
            strain_gauge_data[f'SG{sg_number}_sigma_1 [MPa]'], strain_gauge_data[f'SG{sg_number}_sigma_2 [MPa]'] = principal_stresses.T
            strain_gauge_data[f'SG{sg_number}_theta_p [°]'] = principal_strain_orientation
            strain_gauge_data[f'SG{sg_number}_Biaxiality_Ratio'] = biaxiality_ratios
            strain_gauge_data[f'SG{sg_number}_von_Mises [MPa]'] = von_mises_stresses
        else:
            print(f'Angles for Rosette {sg_number} not found in angles file.')
    else:
        print(f'Unexpected number of columns for Rosette {sg_number}.')


----
##### Create an excel file

In [ ]:
# region
strain_gauge_data.insert(0, 'Time', time)
excel_file_path = "strain_gauge_data_results.xlsx"
strain_gauge_data.to_excel(excel_file_path, index=False)
wb = load_workbook(excel_file_path)
ws = wb.active
columns_to_hide = [col for col in ws.columns if col[0].value is not None and 'von_Mises' not in col[0].value and col[0].value != 'Time']
for column in columns_to_hide:
    ws.column_dimensions[column[0].column_letter].hidden = True
wb.save(excel_file_path)
# endregion
strain_gauge_data

##### Output Columns

In [ ]:
# Checking the data inside the output excel
for column in ws.columns:
        print(f"Header: {column[0].value}, Column: {column[0].column_letter}")


---
#### **Comparison between real measurements & FEA**

In [ ]:
# Plotting the data
# region
import numpy as np
import plotly.graph_objs as go
from scipy.interpolate import interp1d
import ipywidgets as widgets
from IPython.display import display

#import endaq
#endaq.plot.utilities.set_theme(theme='endaq')

# Generate a pure sine wave as one dataset and offset it
x1 = np.linspace(0, 10, 100)
y1 = np.sin(x1) + 1.5

# Generate a slightly randomized sine wave as the other dataset and offset it
x2 = np.linspace(0, 10, 50)
y2 = np.sin(x2) + np.random.normal(0, 0.1, 50) + 1.5

# Create interpolation functions for both datasets
interp_func1 = interp1d(x1, y1, kind='linear', fill_value='extrapolate')
interp_func2 = interp1d(x2, y2, kind='linear', fill_value='extrapolate')

# Create a common set of x values for comparison
common_x = x1

# Calculate the interpolated y values for both datasets
interpolated_y1 = interp_func1(common_x)
interpolated_y2 = interp_func2(common_x)

# Calculate the relative error as a percentage
relative_error_percent = ((interpolated_y2 - interpolated_y1) / interpolated_y1) * 100

# Calculate the difference between the offset sine waves
difference = interpolated_y1 - interpolated_y2

# Initialize the plot as a FigureWidget with subplots
fig = go.FigureWidget(
    data=[
        go.Scatter(x=common_x, y=interpolated_y1, mode='lines', name='y1_Interpolated ', xaxis='x1', yaxis='y1'),
        go.Scatter(x=common_x, y=interpolated_y2, mode='lines', name='y2_Interpolated ', xaxis='x1', yaxis='y1'),
        go.Scatter(x=common_x, y=relative_error_percent, mode='lines', name='Error (%)', xaxis='x2', yaxis='y2'),
        go.Scatter(x=common_x, y=difference, mode='lines', name='Δy', xaxis='x3', yaxis='y3')
    ],
    layout=go.Layout(
        title="Comparison Between Datasets",
        height=800,  # Initial height
        xaxis=dict(domain=[0, 1], anchor='y1'),
        yaxis=dict(domain=[0.68, 1], title='y'),  # Adjust domain for spacing
        xaxis2=dict(domain=[0, 1], anchor='y2', matches='x1'),
        yaxis2=dict(domain=[0.36, 0.64], title='Error(%)'),  # Adjust domain for spacing
        xaxis3=dict(domain=[0, 1], anchor='y3', matches='x1'),
        yaxis3=dict(domain=[0, 0.32], title='Δy'),  # Adjust domain for spacing
        showlegend=True
    )
)

def create_dimension_sliders(fig, initial_height=500, min_height=300, max_height=1500, height_step=50,
                             initial_width=800, min_width=300, max_width=1500, width_step=50):
    """
    Create and display sliders to control the dimensions (height and width) of a Plotly figure.

    Parameters:
    - fig: The figure object to which the dimension sliders will be linked.
    - initial_height: The initial height of the figure.
    - min_height: The minimum height of the figure.
    - max_height: The maximum height of the figure.
    - height_step: The step size for the height adjustment.
    - initial_width: The initial width of the figure.
    - min_width: The minimum width of the figure.
    - max_width: The maximum width of the figure.
    - width_step: The step size for the width adjustment.
    """
    # Function to update the height of the plot
    def update_height(height):
        fig.layout.height = height

    # Function to update the width of the plot
    def update_width(width):
        fig.layout.width = width

    # Function to handle height change
    def on_height_change(change):
        update_height(change.new)

    # Function to handle width change
    def on_width_change(change):
        update_width(change.new)

    # Create a slider widget for total_height
    height_slider = widgets.IntSlider(
        value=initial_height, 
        min=min_height, 
        max=max_height, 
        step=height_step, 
        description='Figure Height'
    )
    
    # Create a slider widget for total_width
    width_slider = widgets.IntSlider(
        value=initial_width, 
        min=min_width, 
        max=max_width, 
        step=width_step, 
        description='Figure Width'
    )
    
    # Display the sliders and the plot
    display(height_slider)
    display(width_slider)
    display(fig)
    
    # Add an observer to the slider to update the plot height and width
    height_slider.observe(on_height_change, names='value')
    width_slider.observe(on_width_change, names='value')

create_dimension_sliders(fig)

# endregion

In [ ]:
if "_measured" in file_path:
    print("do something here")

#strain_gauge_data.columns=strain_gauge_data.iloc[0]
strain_gauge_data.set_index('Time', inplace=True)
strain_gauge_data
#strain_gauge_data.columns[2] = ['Time']
#strain_gauge_datadata = data[1:]
#strain_gauge_data

In [ ]:
px.scatter(strain_gauge_data)

In [ ]:
# Step 1: Check Data Types
data_types = strain_gauge_data.dtypes

# Step 2: Identify Non-Numeric Columns
non_numeric_columns = data_types[data_types != 'int64'][data_types != 'float64'].index.tolist()

non_numeric_columns